In [ ]:
import re
import csv
import numpy as np
import pandas as pd
import inflect
import matplotlib.pyplot as plt

In [ ]:
engine = inflect.engine()

In [ ]:
engine.singular_noun(' ')

In [ ]:
ingredients = open("data/recipeClean/ingredients.txt", mode='r', buffering=-1, encoding="ISO-8859-1", errors=None, newline=None, closefd=True, opener=None)
with open('data/to_remove.csv', 'r') as f:
    reader = csv.reader(f)
    to_remove = [item.lower() for sublist in list(reader) for item in sublist]

In [ ]:
content = ingredients.readlines()

In [ ]:
ids = np.zeros(len(content), dtype=object)
titles = np.zeros(len(content), dtype=object)
ingredients = np.zeros(len(content), dtype=object)
for i in range(len(content)):
    line = content[i].split('\t')
    if len(line) > 4:
        id_recipe, title, ingredient = line[0], line[3], line[4]
        ids[i] = id_recipe
        titles[i] = title
        ingredients[i] = ingredient
    else:
        ids[i] = line[0]
        titles[i] = line[2]
        ingredients[i] = ""

In [ ]:
titles_pd = pd.DataFrame(titles)
titles_pd = titles_pd.rename(columns={'0':'Titles'})
titles_pd.head(5)

In [ ]:
ingredients_pd = pd.DataFrame(ingredients)

In [ ]:
def split_ingredient(ingredient):
    return ingredient.split('|')

In [ ]:
def clean_ingredient(ingredient):
    ingredient = remove_par(ingredient)
    ingredient = remove_special_char(ingredient)
    ingredient = remove_letter(ingredient)
    ingredient = remove_number(ingredient)
    ingredient = remove_whitespace_comma(ingredient)
    ingredient = remove_useless_words(ingredient)
    ingredient = remove_adjective(ingredient)
    ingredient = remove_alternative(ingredient)
    ingredient = ingredient.lower()
    ingredient = remove_minus(ingredient)
    ingredient = remove_conjonction(ingredient)
    ingredient = remove_space(ingredient)
    if engine.singular_noun(ingredient):
        return engine.singular_noun(ingredient)
    else:
        return ingredient
    
def remove_par(ingredient):
    return re.sub(r" \([^)]*\)","", ingredient)

def remove_special_char(ingredient):
    return re.sub(r'/|\n|%|:|\*|\.|#' , '', ingredient)

def remove_letter(ingredient):
    ingredient = re.sub(r'(\A| )s( |\Z)' , ' ', ingredient)
    ingredient = re.sub(r'(\A| )(t|T)( |\Z)' , ' ', ingredient)
    ingredient = re.sub(r'(\A| )c( |\Z)' , ' ', ingredient)
    ingredient = re.sub(r'(\A| )a ' , ' ', ingredient)
    ingredient = re.sub(r'(\A| )g ' , ' ', ingredient)
    return ingredient

def remove_number(ingredient):
    return re.sub(r"\d+", '', ingredient)

def remove_whitespace_comma(ingredient):
    return ingredient.lstrip().split(',')[0]

def remove_useless_words(ingredient):
    ingredient_list = ingredient.split(' ')
    ingredient_list = [word for word in ingredient_list if (word not in to_remove)]
    ingredient = ' '.join(ingredient_list)
    return ingredient

def remove_adjective(ingredient):
    return re.sub(r'\w+ed\s','',ingredient)

def remove_alternative(ingredient):
    return ingredient.split(' or ')[0]

def remove_minus(ingredient):
    return re.sub(r'\A-', '', ingredient)

def remove_conjonction(ingredient):
    return re.sub(r'\Aof |\Aand |\Ato ', '', ingredient)

def remove_space(ingredient):
    ingredient = re.sub(r'( )+', ' ', ingredient)
    return re.sub(r'\A ', '', ingredient)

In [ ]:
list(map(lambda x: clean_ingredient(x), split_ingredient(ingredients[0])))

In [ ]:
#for ingredient in ingredients:
#    print(list(map(lambda x: clean_ingredient(x), split_ingredient(ingredient))))

In [ ]:
"""for i in range(len(ingredients)):
    print("cleaned: ")
    print(list(map(lambda x: clean_ingredient(x), split_ingredient(ingredients[i]))))
    print("original: ")
    print(ingredients[i])
    #a = split_ingredient(ingredients[i])
    #for b in a:  
    #    if('tb' in b):
    #        print(b)
    #        print('\n')"""

In [ ]:
clean_recipes = []
for i in range(len(ingredients)):
    clean_recipe = list(map(lambda x: clean_ingredient(x), split_ingredient(ingredients[i])))
    clean_recipe = [x for x in clean_recipe if x != '' or x != '\'' or x != '\\n' or x != ['']]
    if clean_recipe:
        clean_recipes.append(clean_recipe)

In [ ]:
len(clean_recipes)

In [ ]:
ids_titles_recipes = list(zip(ids, titles, clean_recipes))

In [ ]:
ids_titles_recipes = [x for x in ids_titles_recipes if (x[2] != [''])]

In [ ]:
ids_clean = np.array([x[0] for x in ids_titles_recipes])
titles_clean = np.array([x[1] for x in ids_titles_recipes])
recipes_clean = np.array([x[2] for x in ids_titles_recipes])

In [ ]:
df = pd.DataFrame({'id': ids_clean, 'title': titles_clean, 'recipe': recipes_clean})
df

In [ ]:
df.to_json("clean_cookies_recipes.json")

In [ ]:
df_recipe = df['recipe'].apply(pd.Series).stack().to_frame().reset_index()
df_test = df['recipe']
df_recipe = df_recipe[0].to_frame()

In [ ]:
df_recipe

In [ ]:
recipes_list = list(df_test.values)
recipes_list
b = pd.DataFrame(recipes_list)
b

In [ ]:
df_recipe = df_recipe.rename(columns={0: 'ingredient'})

In [ ]:
df_unique = df_recipe['ingredient'].unique()
len(df_unique)

In [ ]:
df_test

In [ ]:
ls = list()
for i in df_test:
    for ingr in i:
        ls.append(ingr)

In [ ]:
se = pd.Series(ls)

In [ ]:
count_table = se.value_counts().to_frame()
#count_table[count_table[0] > 10]
count_table.hist(bins=500, figsize=(15, 8))
plt.show()

In [ ]:
a = count_table.reset_index()
print(a[a['index'] == 'cucumbers'])
print('\n')
print(a[a['index'] == 'cucumber'])
print('\n')
print(a[a['index'] == 'tomato'])
print('\n')
print(a[a['index'] == 'tomatoes'])
print('\n')
print(a[a['index'] == 'eggs'])
print('\n')
print(a[a['index'] == 'egg'])

In [ ]:
count_table.plot(logy=True)
plt.show()